# ASD analysis - variability v1

Take windows by 1 (2, 4) seconds. Compute power in some band (alpha, theta?). Compute variance

In [127]:
from os.path import join
import mne
import pandas as pd
import numpy as np

from scipy.stats import ttest_ind
from sklearn.metrics import roc_auc_score

from statsmodels.stats.multitest import multipletests

pd.set_option('display.max_rows', 1000)

In [30]:
%matplotlib notebook

In [2]:
path = '../../preproc_data/asd/'

In [119]:
path_df = pd.read_csv(join(path, 'path_file.csv'))

Вопросы

* dekanev_sven_19.raw.fif - нет возраста
* 01_михна_степан12_19.raw.fif - ошибка
* komleva_org_asd_19.raw.fif - нет возраста
* serega_org_15_19.raw.fif - точно typical?
* alex5_male_fon_19.raw.fif - нет О2?
* arkasha_5_asd2304_og_19.raw.fif - нет T5

In [147]:
to_exclude = [
    'dekanev_sven_19.raw.fif',
    '01_михна_степан12_19.raw.fif',
    'komleva_org_asd_19.raw.fif',
    'serega_org_15_19.raw.fif',
    'alex5_male_fon_19.raw.fif',
    'arkasha_5_asd2304_og_19.raw.fif'
]

In [148]:
path_df = path_df.loc[~path_df['fn'].isin(to_exclude)]

In [121]:
path_df.head()

,fn,target,dataset_name,sfreq,age,seconds
0,sedrykyn_sasha_7_og_concat_19.raw.fif,asd,asd,125,7,47.0
1,roma gritchin _5_fon_open_19.raw.fif,asd,asd,125,5,33.0
2,boy5_asd_og_new_19.raw.fif,asd,asd,125,5,50.0
3,viflyancev_4_asd_fon__concat_19.raw.fif,asd,asd,125,4,58.0
4,andrey_matveev3_asd_new_19.raw.fif,asd,asd,125,3,50.0


In [104]:
def compute_variability(ar, batch_size=125):
    n = len(ar)
    cur_idx = 0
    power_ = []
    while (cur_idx + 1) * batch_size < n:
        start_idx = cur_idx * batch_size
        end_idx = (cur_idx + 1) * batch_size
        power_.append((ar[start_idx:end_idx] ** 2).sum())
        cur_idx += 1
    power_ = np.array(power_)
    return power_.mean(), power_.std(), power_.std() / power_.mean()

In [105]:
def get_variability_features(raw, channels, window_len=1):
    batch_size = int(window_len * raw.info['sfreq'])
    raw = raw.copy()
    raw.filter(4, 12)
    d = {}
    for ch in channels:
        mean_, std_, variabty_ = compute_variability(raw[ch][0].ravel(), batch_size=batch_size)
        d[ch + '_mean'] = mean_
        d[ch + '_std'] = std_
        d[ch + '_variabty'] = variabty_
    return d

In [150]:
groups = [
    ('2-4', 2, 4),
    ('5-6', 5, 6),
    ('7+', 7, 100),
]

for g in groups:
    path_df.loc[(path_df['age'] >= g[1]) & (path_df['age'] <= g[2]), 'age_group'] = g[0]

In [152]:
channels = [
 'Fp1',
 'Fp2',
 'F7',
 'F3',
 'Fz',
 'F4',
 'F8',
 'T3',
 'C3',
 'Cz',
 'C4',
 'T4',
 'T5',
 'P3',
 'Pz',
 'P4',
 'T6',
 'O1',
 'O2']

res_rows = []
for i, row in path_df.iterrows():
    raw = mne.io.read_raw_fif(join(path, row['fn']), preload=True, verbose=False)
    d = get_variability_features(raw, channels, window_len=1)
    d['fn'] = row['fn']
    res_rows.append(d)
res = pd.DataFrame(res_rows)

In [155]:
df = res.merge(path_df[['fn', 'age_group', 'target']], on='fn')

In [156]:
features = [col for col in df.columns if col not in ['fn', 'age_group', 'target']]

In [132]:
df['target'].value_counts()

typical    186
asd        152
Name: target, dtype: int64

In [133]:
def feat_performance(df, features=None):
    rows = []
    df_0 = df[df['target'] == 'typical'].copy()
    df_1 = df[df['target'] == 'asd'].copy()
    
    for feat in features:        
        
        mean_diff = df_0[feat].mean() - df_1[feat].mean()
        mean_ratio = df_0[feat].mean() / df_1[feat].mean()
        ttest_res = ttest_ind(df_0[feat], df_1[feat], equal_var=False)
        roc_auc = max(roc_auc_score(df['target'], df[feat]), 1 - roc_auc_score(df['target'], df[feat]))
        
        d = {
            'feature': feat,
            'ttest_pval': ttest_res.pvalue,
            'ttest_stat': ttest_res.statistic,
            'mean_hc': df_0[feat].mean(),
            'mean_asd': df_1[feat].mean(),
            'roc_auc': roc_auc,
            'mean_diff': mean_diff,
            'mean_ratio': mean_ratio,
        }
        rows.append(d)
    res = pd.DataFrame(rows)
    return res

In [164]:
feat_performance(df, features).sort_values('ttest_pval').head(10)

,feature,ttest_pval,ttest_stat,mean_hc,mean_asd,roc_auc,mean_diff,mean_ratio
41,P3_variabty,0.094917,1.675095,6.366709e-01,0.568249,0.552482,0.068422,1.120408
32,C4_variabty,0.152051,1.435643,5.920062e-01,0.536715,0.565549,0.055292,1.103018
26,C3_variabty,0.181339,1.339439,6.216555e-01,0.559152,0.587446,0.062504,1.111783
5,Fp2_variabty,0.238614,1.180572,6.262216e-01,0.562660,0.545503,0.063561,1.112966
49,T6_std,0.318693,-1.000472,2.042834e-09,0.000008,0.557716,-0.000008,0.000262
55,O2_std,0.318712,-1.000434,2.386549e-09,0.000008,0.530691,-0.000008,0.000294
43,Pz_std,0.318824,-1.000201,4.216556e-09,0.000021,0.556647,-0.000021,0.000203
46,P4_std,0.318828,-1.000193,3.829743e-09,0.000012,0.564623,-0.000012,0.000331
1,Fp1_std,0.318831,-1.000187,2.863257e-09,0.000013,0.565406,-0.000013,0.000227
52,O1_std,0.318833,-1.000183,2.190480e-09,0.000008,0.540554,-0.000008,0.000270


In [160]:
feat_performance(df[df['age_group'] == '2-4'], features).sort_values('ttest_pval').head(10)

,feature,ttest_pval,ttest_stat,mean_hc,mean_asd,roc_auc,mean_diff,mean_ratio
18,F8_mean,0.029966,-2.211573,4.670385e-09,7.147496e-09,0.607201,-2.477110e-09,0.653430
19,F8_std,0.039476,-2.098240,1.995023e-09,3.098139e-09,0.602837,-1.103116e-09,0.643942
36,T5_mean,0.055041,1.960260,4.816628e-09,3.017645e-09,0.554283,1.798982e-09,1.596154
1,Fp1_std,0.060789,-1.911396,2.511162e-09,4.390448e-09,0.591380,-1.879286e-09,0.571960
3,Fp2_mean,0.069121,-1.841390,5.301105e-09,7.565595e-09,0.596836,-2.264490e-09,0.700686
41,P3_variabty,0.070592,1.843460,6.216159e-01,4.996212e-01,0.600655,1.219946e-01,1.244174
15,F4_mean,0.073123,-1.816363,6.853149e-09,9.688456e-09,0.582651,-2.835307e-09,0.707352
0,Fp1_mean,0.074362,-1.807103,5.536507e-09,7.767893e-09,0.597927,-2.231386e-09,0.712742
53,O1_variabty,0.135130,1.519838,6.227963e-01,5.096045e-01,0.545554,1.131918e-01,1.222117
16,F4_std,0.141872,-1.484573,3.024865e-09,4.093151e-09,0.562466,-1.068286e-09,0.739006


In [161]:
feat_performance(df[df['age_group'] == '5-6'], features).sort_values('ttest_pval').head(10)

,feature,ttest_pval,ttest_stat,mean_hc,mean_asd,roc_auc,mean_diff,mean_ratio
54,O2_mean,0.006666,-2.853477,2.929198e-09,6.550545e-09,0.654842,-3.621347e-09,0.447169
51,O1_mean,0.008606,-2.758890,2.547859e-09,5.610718e-09,0.661599,-3.062859e-09,0.454106
42,Pz_mean,0.018504,-2.415299,6.501235e-09,1.031071e-08,0.674550,-3.809479e-09,0.630532
45,P4_mean,0.023946,-2.308465,6.029797e-09,8.993231e-09,0.654842,-2.963433e-09,0.670482
30,C4_mean,0.026463,-2.276159,6.029176e-09,8.882078e-09,0.618243,-2.852902e-09,0.678802
3,Fp2_mean,0.030108,-2.228734,4.844833e-09,7.649222e-09,0.614302,-2.804389e-09,0.633376
0,Fp1_mean,0.031565,-2.204597,4.940856e-09,7.447149e-09,0.610360,-2.506293e-09,0.663456
4,Fp2_std,0.031711,-2.220746,2.474653e-09,4.294536e-09,0.585586,-1.819882e-09,0.576233
55,O2_std,0.031820,-2.227532,1.550531e-09,4.456730e-09,0.672297,-2.906198e-09,0.347908
14,Fz_variabty,0.033541,2.161887,5.939130e-01,4.764552e-01,0.617680,1.174578e-01,1.246524


In [162]:
feat_performance(df[df['age_group'] == '7+'], features).sort_values('ttest_pval').head(10)

,feature,ttest_pval,ttest_stat,mean_hc,mean_asd,roc_auc,mean_diff,mean_ratio
47,P4_variabty,0.085982,1.729422,7.295835e-01,0.592407,0.566561,0.137176,1.231557
26,C3_variabty,0.213567,1.248672,6.844174e-01,0.591479,0.584351,0.092939,1.157129
11,F3_variabty,0.215306,1.244053,6.314706e-01,0.544336,0.557666,0.087135,1.160076
35,T4_variabty,0.259815,1.131001,7.493097e-01,0.613061,0.534901,0.136249,1.222244
41,P3_variabty,0.315711,1.006681,6.575947e-01,0.594782,0.542741,0.062812,1.105606
49,T6_std,0.320756,-1.000431,2.245021e-09,0.000018,0.540781,-0.000018,0.000128
55,O2_std,0.320791,-1.000357,2.384216e-09,0.000018,0.534750,-0.000018,0.000130
43,Pz_std,0.320884,-1.000164,4.179472e-09,0.000047,0.568973,-0.000047,0.000089
7,F7_std,0.320903,-1.000126,3.303628e-09,0.000024,0.526459,-0.000024,0.000140
1,Fp1_std,0.320903,-1.000124,3.182788e-09,0.000028,0.545002,-0.000028,0.000112
